In [1]:
import pandas as pd
import numpy as np

#para la extracción de datos
import requests  # Requests permite hacer peticiones HTTP para interactuar con APIs y sitios web.
import selenium
from selenium import webdriver  # WebDriver permite controlar un navegador de forma automatizada.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait  # WebDriverWait permite esperar hasta que una condición específica se cumpla.
from selenium.webdriver.support import expected_conditions as EC  # EC ayuda a definir condiciones para WebDriverWait.
from selenium.common.exceptions import NoSuchElementException  # Excepciones comunes de Selenium que nos podemos encontrar.
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup  # BeautifulSoup permite parsear HTML y extraer información del contenido de las páginas web.
from time import sleep
import re
import random

import sys 
import os
import warnings
warnings.filterwarnings("ignore")
sys.path.append(os.path.abspath("../../src"))   
import soporte_extraccion as q
pd.set_option('display.max_columns', None)

Hacer extracción con selenium

In [4]:
driver = webdriver.Chrome()
url_red = "https://www.redpiso.es/venta-viviendas/barcelona/pagina-1"
driver.get(url_red)
sleep(3)

# #rechazamos las cookies
# driver.find_element("css selector", "#gdpr-cookie-reject").click()
# sleep(2)

# #pinchamos el primer anuncio
# driver.find_element("css selector", "#row_RP2072023114464 > div > div").click()

# #cojo las caracterstcas
# caracteristicas=driver.find_element("css selector", "body > section:nth-child(13) > div > div > div:nth-child(2)").text

# #cojo el precio
# precio=driver.find_element("css selector", "body > section:nth-child(7) > div > div > div > div > div > div > div > div.property-contact-item > h2").text



# #pinchamos el segundo anuncio
# driver.find_element("css selector", "#row_RP2072024132638 > div > div").click()

# #cojo las caracterstcas2
# caracteristicas2=driver.find_element("css selector", "body > section:nth-child(13) > div > div > div:nth-child(2)").text

# #cojo el precio2
# precio2=driver.find_element("css selector", "body > section:nth-child(7) > div > div > div > div > div > div > div > div.property-contact-item > h2").text


# #pinchamos el tercer anuncio
# driver.find_element("css selector", "#row_RP2012024131598 > div > div").click()

# #cojo las caracterstcas3
# caracteristicas3=driver.find_element("css selector", "body > section:nth-child(13) > div > div > div:nth-child(2)").text

# #cojo el precio3
# precio3=driver.find_element("css selector", "body > section:nth-child(7) > div > div > div > div > div > div > div > div.property-contact-item > h2").text

# driver.back()




#esta es toda la tabla de contenido
#    <div id="ppt_list">

#esto es cada articulo
#     <article>

#esta es la parte tendría que sacar con bs4
#    <div id="row_RP2072023113582" class="col-lg-3 col-md-4 col-sm-6 col-xs-12">   




#IDEA DE DAVID COGER LOS LINKS DE CADA ANUNCIO(CON bs4)
#    <a class="item-link" data-id-property="RP2012024131598" href="https://www.redpiso.es/inmueble/piso-en-venta-en-hospitalet-de-llobregat-barcelona-RP2012024131598">Piso en venta en Hospitalet de Llobregat, Barcelona</a>

Quizá dado que me estoy fijando en que los id no cambian al refresacar pero si enre los distintos anuncios, igual puedo usar bs4 para sacar esos ids e iterra por ellos para ir sacando los precios y caraterísticas de cada vivienda

In [30]:
res= requests.get(url_red)
sopa_pisos= BeautifulSoup(res.content, "html.parser")
pisos= sopa_pisos.find_all("a", class_ = "item-link" )

In [43]:
driver = webdriver.Chrome()
lista_links=[]
df_intermedio=pd.DataFrame()
for link_sucio in pisos:
    links= link_sucio.get("href")
    lista_links.append(links)
    
for link in lista_links:
    driver.get(link)
    caracteristicas_prueba=driver.find_element("css selector", "body > section:nth-child(13) > div > div > div:nth-child(2)").text

    #cojo el precioprueba
    precio_prueba=driver.find_element("css selector", "body > section:nth-child(7) > div > div > div > div > div > div > div > div.property-contact-item > h2").text
    coste=precio_prueba.replace(".","").replace(" €","")
    datos=caracteristicas_prueba.replace(":","").split("\n")
    datos.append(coste)
    df=pd.DataFrame(datos).T
    df_intermedio=pd.concat([df_intermedio,df], axis=0, ignore_index=True)


BIEN PERO TENGO QUE VER COMO JUNTARLOS Y QUE LOS NULOS SE QUEDEN DONDE DEBERÍAN (TRY EXCEPT POR EJEMPLO)

In [6]:
res = requests.get(url_red)
sopa_pisos = BeautifulSoup(res.content, "html.parser")
pisos = sopa_pisos.find_all("a", class_="item-link")
driver = webdriver.Chrome()

lista_links = []
df_intermedio = pd.DataFrame()
columnas_globales = set()

# Obtener todos los enlaces de los anuncios
for link_sucio in pisos:
    links = link_sucio.get("href")
    lista_links.append(links)

#Uso regex para detectar las claves con :
regex_clave_valor = r"([^:]+):\s*(.+)"  # siguiendo la estructura "clave: valor"

for link in lista_links:
    driver.get(link)
    
    #Obtengo las características y precio

    caracteristicas_prueba = driver.find_element("css selector","body > section:nth-child(13) > div > div > div:nth-child(2)").text
    
    precio_prueba = driver.find_element("css selector","body > section:nth-child(7) > div > div > div > div > div > div > div > div.property-contact-item > h2").text
    
    #proceso el precio
    coste = precio_prueba.replace(".", "").replace(" €", "")
    
    #proceso las características
    datos = caracteristicas_prueba.replace(" m²","").split("\n")
    datos.append(f"Precio: {coste}")  # apendeo el precio pero metiendole la misma extructura de clave: valor que el resto
    
    caracteristicas_dict = {}
    
    for dato in datos:
        match = re.match(regex_clave_valor, dato)
        if match:
            clave = match.group(1).strip()             # lo de antes de ':'
            valor = match.group(2).strip()             # lo de después de ':'
            caracteristicas_dict[clave] = valor
        else:
            
            caracteristicas_dict[dato.strip()] = "Si"  #para casos como exterior o ascensor que no tienen como tal clave(porque no tiene :) añadir como valor Si

    #genero el df del dicc
    df_anuncio = pd.DataFrame([caracteristicas_dict])
    
    #actualizo la lista global de columnas únicas
    columnas_globales.update(df_anuncio.columns)
    
    #coloco los valores en las columnas donde correspondan rellenando con None
    df_anuncio = df_anuncio.reindex(columns=columnas_globales, fill_value=None)
    
    #concateno al dataframe intermedio
    df_intermedio = pd.concat([df_intermedio, df_anuncio], ignore_index=True)



WebDriverException: Message: unknown error: cannot determine loading status
from target frame detached
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF6425EFB05+28789]
	(No symbol) [0x00007FF6425586E0]
	(No symbol) [0x00007FF6423F5759]
	(No symbol) [0x00007FF6423E35FB]
	(No symbol) [0x00007FF6423E1274]
	(No symbol) [0x00007FF6423E1D1F]
	(No symbol) [0x00007FF6423F127E]
	(No symbol) [0x00007FF642404AAD]
	(No symbol) [0x00007FF64240B27A]
	(No symbol) [0x00007FF6423E2484]
	(No symbol) [0x00007FF6424045FA]
	(No symbol) [0x00007FF64248F99B]
	(No symbol) [0x00007FF64246F1E3]
	(No symbol) [0x00007FF64243A938]
	(No symbol) [0x00007FF64243BAA1]
	GetHandleVerifier [0x00007FF64292933D+3410093]
	GetHandleVerifier [0x00007FF64293E7DD+3497293]
	GetHandleVerifier [0x00007FF642932A73+3448803]
	GetHandleVerifier [0x00007FF6426B7BBB+848171]
	(No symbol) [0x00007FF642563C3F]
	(No symbol) [0x00007FF64255F6E4]
	(No symbol) [0x00007FF64255F87D]
	(No symbol) [0x00007FF64254ED

In [12]:
df_intermedio

,Dormitorios,Estado,Metros,Orientación,Cal. energética:,Baños,Metros útiles,Antigüedad,Tipo de fachada,Precio,Exterior,Año de construcción,Tipo de suelo,Número de terrazas,Aire acondicionado,Alarma,Pisos,Consumo,Emisiones,Plantas,Cocina equipada,Agua caliente,Puerta de seguridad,Armarios,Ascensor,Acceso minusválidos,Calefacción,Amueblado,Circuito cerrado de seguridad
0,2,A reformar,30 m²,Noreste,Si,1,24 m²,Más de 50 años,Cemento,89000,Si,1870,Gres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Perfecto Estado,38 m²,NaN,Si,1,32 m²,Entre 35 y 50 Años,Ladrillo,100000,Si,1957,Gres,1,Si,Si,8,149 KW h/m² año,30 Kg CO2/m² año,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Perfecto Estado,43 m²,NaN,Si,1,NaN,Más de 50 años,Otros materiales,130000,NaN,1932,Gres,NaN,NaN,NaN,NaN,120 KW h/m² año,90 Kg CO2/m² año,NaN,Si,Gas natural,Si,NaN,NaN,NaN,NaN,NaN,NaN
3,2,Perfecto Estado,60 m²,Suroeste,Si,1,43 m²,Más de 50 años,NaN,145000,NaN,NaN,Terrazo,1,Si,NaN,6,NaN,NaN,6,NaN,Individual,Si,1,Si,Si,Gas Natural,NaN,NaN
4,1,Perfecto Estado,59 m²,Norte,Si,1,49 m²,Más de 50 años,Cemento,156000,NaN,1936,Parquet,NaN,Si,Si,NaN,NaN,NaN,NaN,Si,NaN,NaN,NaN,NaN,NaN,Eléctrica,Si,NaN
5,2,Perfecto Estado,52 m²,Sur,Si,1,43 m²,Más de 50 años,Piedra,170000,Si,1960,Parquet,NaN,NaN,NaN,1,185 KW h/m² año,48 Kg CO2/m² año,NaN,Si,Gas natural,Si,NaN,NaN,NaN,NaN,NaN,NaN
6,1,Perfecto Estado,54 m²,Sur,NaN,1,45 m²,Entre 35 y 50 Años,Ladrillo,179500,NaN,1964,Gres,NaN,Si,NaN,32,NaN,NaN,8,Si,Individual,NaN,NaN,Si,NaN,NaN,Si,Si
7,4,Perfecto Estado,141 m²,NaN,Si,2,130 m²,Más de 50 años,Otros materiales,199000,Si,1972,Parquet,NaN,Si,NaN,NaN,120 KW h/m² año,90 Kg CO2/m² año,NaN,Si,Gas natural,Si,NaN,NaN,NaN,Individual,NaN,NaN
8,3,Perfecto Estado,64 m²,Noroeste,Si,1,54 m²,Entre 35 y 50 Años,Ladrillo,220000,Si,1960,Gres,1,NaN,NaN,10,103 KW h/m² año,21 Kg CO2/m² año,5,Si,Individual,NaN,NaN,NaN,NaN,Gas Natural,NaN,NaN
9,3,Perfecto Estado,60 m²,Noreste,Si,1,53 m²,Entre 35 y 50 Años,Ladrillo,239000,NaN,1970,Gres,1,NaN,NaN,20,156 KW h/m² año,33 Kg CO2/m² año,5,Si,Individual,NaN,NaN,Si,NaN,NaN,NaN,NaN


In [31]:
pisos

[<a class="item-link" data-id-property="RP2072023114464" href="https://www.redpiso.es/inmueble/piso-en-venta-en-calle-carretes-el-raval-ciutat-vella-barcelona-barcelona-RP2072023114464">      Piso en venta en CALLE Carretes, El Raval, Ciutat Vella, Barcelona, Barcelona</a>,
 <a class="item-link" data-id-property="RP2072024132638" href="https://www.redpiso.es/inmueble/piso-en-venta-en-calle-pere-galvany-esplugues-de-llobregat-barcelona-RP2072024132638">      Piso en venta en CALLE Pere Galvany, Esplugues de Llobregat, Barcelona</a>,
 <a class="item-link" data-id-property="RP2012024131598" href="https://www.redpiso.es/inmueble/piso-en-venta-en-hospitalet-de-llobregat-barcelona-RP2012024131598">Piso en venta en Hospitalet de Llobregat, Barcelona</a>,
 <a class="item-link" data-id-property="RP2012022105801" href="https://www.redpiso.es/inmueble/piso-en-venta-en-santa-coloma-de-gramenet-barcelona-RP2012022105801">Piso en venta en Santa Coloma de Gramenet, Barcelona</a>,
 <a class="item-link

In [27]:
encabezados=["tamanio", "metros_utiles","dormitorios", "banios","calidad_energetica","estado","orientacion","ventanas","tipo_suelo","tipo_fachada","antiguedad","anio_construccion","precio"]

In [13]:
#habrá que cambiar Tenerife por Santa Cruz de Tenerife antes de pasar el modelo predictivo
lista_prov1=['Albacete', 'Alicante', 'Ávila',
       'Barcelona', 'Burgos', 'Cádiz', 'Castellón',
       'Córdoba', 'Cuenca',
       'Granada', 'Guadalajara', 'Huelva', 'Huesca', 'Lleida', 'Madrid', 'Málaga', 'Murcia',
       'Ourense', 'Palencia',  'Segovia', 'Sevilla',"Tenerife",
       'Toledo', 'Valencia', 'Valladolid', 'Zaragoza']
len(lista_prov1)

26

In [ ]:
lista_prov2=['Albacete', 'Alicante', "Almería", 'Ávila', "Badajoz",
       'Barcelona', 'Burgos', 'Cádiz', 'Castellón',
       'Córdoba', 'Cuenca',
       'Granada', 'Guadalajara', 'Huelva', 'Huesca', 'Lleida', 'Madrid', 'Málaga', 'Murcia',
       'Ourense', 'Palencia', "Pontevedra", 'Segovia', 'Sevilla', "Tarragona","Tenerife",
       'Toledo', 'Valencia', 'Valladolid', 'Zaragoza']

In [3]:
lista_prov3=["Álava", 'Albacete', 'Alicante', "Almería", 'Ávila', "Badajoz",
       "Baleares", 'Barcelona', 'Burgos', "Cáceres", 'Cádiz', 'Castellón',
       "Ciudad Real", 'Córdoba', "A Coruña", 'Cuenca', "Girona",
       'Granada', 'Guadalajara', "Guipúzcoa", 'Huelva', 'Huesca', "Jaén",
       "León", 'Lleida', "La Rioja", "Lugo", 'Madrid', 'Málaga', 'Murcia',
       "Navarra", 'Ourense', "Asturias", 'Palencia',
       "Las Palmas de Gran Canaria", "Pontevedra", "Salamanca",
       "Tenerife", "Cantabria", 'Segovia', 'Sevilla',
       "Soria", "Tarragona", "Teruel", 'Toledo', 'Valencia', 'Valladolid',
       "Vizcaya", "Zamora", 'Zaragoza', "Ceuta", "Melilla"]

lista_provincias=[]
for prov in lista_prov3:
      provincia=prov.replace(" ","-").lower()
      lista_provincias.append(provincia)


In [4]:


columnas_globales = set()
for provincia in lista_provincias:

    df_intermedio_prov=pd.DataFrame()
    for pag in range(1,30):
        url_red =f"https://www.redpiso.es/venta-viviendas/{provincia}/pagina-{pag}"   #en madrid hay 277 pags q equivalen a 3000 y pico hogares pero valencia, malaga etc tienen entorno a 25 pags con unas 300 viviendas cada
        sleep(random.uniform(2, 5))
        try:    
                        
            res = requests.get(url_red)
            sopa_pisos = BeautifulSoup(res.content, "html.parser")
            pisos = sopa_pisos.find_all("a", class_="item-link")
            driver = webdriver.Chrome()

                     

            lista_links = []
            df_intermedio_pag = pd.DataFrame()
            

            # Obtener todos los enlaces de los anuncios
            for link_sucio in pisos:
                links = link_sucio.get("href")
                lista_links.append(links)

            #Uso regex para detectar las claves con :
            regex_clave_valor = r"([^:]+):\s*(.+)"  # siguiendo la estructura "clave: valor"

            for link in lista_links:
                sleep(2)
                driver.get(link)
                
                #Obtengo las características y precio

                caracteristicas_prueba = driver.find_element("css selector","body > section:nth-child(13) > div > div > div:nth-child(2)").text
                
                precio_prueba = driver.find_element("css selector","body > section:nth-child(7) > div > div > div > div > div > div > div > div.property-contact-item > h2").text
                
                #proceso el precio
                coste = precio_prueba.replace(".", "").replace(" €", "")
                
                #proceso las características
                datos = caracteristicas_prueba.replace(" m²","").split("\n")
                datos.append(f"Precio: {coste}")  # apendeo el precio pero metiendole la misma extructura de clave: valor que el resto
                print(datos)
                caracteristicas_dict = {}
                
                for dato in datos:
                    match = re.match(regex_clave_valor, dato)
                    if match:
                        clave = match.group(1).strip()             # lo de antes de ':'
                        valor = match.group(2).strip()             # lo de después de ':'
                        caracteristicas_dict[clave] = valor
                    else:
                        
                        caracteristicas_dict[dato.strip()] = "Si"  #para casos como exterior o ascensor que no tienen como tal clave(porque no tiene :) añadir como valor Si

                #genero el df del dicc
                df_anuncio = pd.DataFrame([caracteristicas_dict])

                df_anuncio["provincia"]= provincia
                print(df_anuncio.columns)
                #actualizo la lista global de columnas únicas
                columnas_globales.update(df_anuncio.columns)
                
                #coloco los valores en las columnas donde correspondan rellenando con None
                df_anuncio = df_anuncio.reindex(columns=columnas_globales, fill_value=None)
                
                #concateno al dataframe intermedio_pag
                df_intermedio_pag = pd.concat([df_intermedio_pag, df_anuncio], ignore_index=True)

                #Voy guardandolos en csv por si me bloquean
                df_intermedio_pag.to_csv(f"../../datos/red_piso/paginas/datos_{provincia}{pag}.csv")

                #concateno al dataframe intermedio_prov
                df_intermedio_prov = pd.concat([df_intermedio_prov, df_intermedio_pag], ignore_index=True)
                # driver.quit()


        except:
            print("ha saltado al except")
            df_intermedio_prov.to_csv(f"../../datos/red_piso/provincias/datos_{provincia}{pag}.csv")
            driver.quit()
            break      #pasar a la página 1 de la siguiente provincia
            





KeyboardInterrupt: 

In [5]:
caracteristicas_prueba

NameError: name 'caracteristicas_prueba' is not defined

In [ ]:
driver = webdriver.Chrome()
for provincia in lista_provincias:
    for pag in range(1,30):
        try:    
            url_red =f"https://www.redpiso.es/venta-viviendas/{provincia}/pagina-{pag}"   #en madrid hay 277 pags q equivalen a 3000 y pico hogares pero valencia, malaga etc tienen entorno a 25 pags con unas 300 viviendas cada
            driver.get(url_red)
            sleep(3)

        except:
            pass
            #pasar a la página 1 de la siguiente provincia (break, pass o continue)


#Que no se me olvide ir guandandolos en csv por si me bloquean
df=pd.DataFrame()
df.to_csv(f"../../datos/red_piso/datos_{provincia}{pag}")

chatti

In [28]:


columnas_globales = set()
driver = webdriver.Chrome()
df_final_prov=pd.DataFrame()
try:
    for provincia in lista_provincias:
        df_intermedio_prov = pd.DataFrame()
        for pag in range(1, 279):    #pongo 279 porque madrid que es la que más páginas tienen tienen 277 páginas
            url_red = f"https://www.redpiso.es/venta-viviendas/{provincia}/pagina-{pag}"
            sleep(random.uniform(2, 5))
            lista_links = []
            try:
                driver.get(url_red)    #lo inicio con selenium en vez de con requests porque causaba fallos
                html = driver.page_source
                sopa_pisos = BeautifulSoup(html, "html.parser")
                pisos = sopa_pisos.find_all("a", class_="item-link")

                if not pisos:
                    print(f"No se encontraron pisos en la página {pag} de la provincia {provincia}.")
                    break  #pasamos a la siguiente provincia
                
                df_intermedio_pag = pd.DataFrame()

                #obtengo los enlaces limpios de los anuncios
                for link_sucio in pisos:
                    links = link_sucio.get("href")
                    
                    lista_links.append(links)

                #uso regex para detectar las claves con :
                regex_clave_valor = r"([^:]+):\s*(.+)"     #siguiendo la estructura "clave: valor"

                for link in lista_links:
                    sleep(random.uniform(1.5, 4.5))  
                    driver.get(link)

                    caracteristicas_prueba = driver.find_element("css selector", "body > section:nth-child(13) > div > div > div:nth-child(2)").text
                    precio_prueba = driver.find_element("css selector", "body > section:nth-child(7) > div > div > div > div > div > div > div > div.property-contact-item > h2").text

                    #proceso el precio
                    coste = precio_prueba.replace(".", "").replace(" €", "")

                    #proceso las características
                    datos = caracteristicas_prueba.replace(" m²", "").split("\n")
                    datos.append(f"Precio: {coste}")
                    caracteristicas_dict = {}

                    for dato in datos:
                        match = re.match(regex_clave_valor, dato)
                        if match:
                            clave = match.group(1).strip()
                            valor = match.group(2).strip()
                            caracteristicas_dict[clave] = valor
                        else:
                            caracteristicas_dict[dato.strip()] = "Si"

                    #genero el df del dicc
                    df_anuncio = pd.DataFrame([caracteristicas_dict])
                    df_anuncio["provincia"] = provincia

                    #actualizo la lista de columnas únicas
                    columnas_globales.update(df_anuncio.columns)

                    #coloco los valores en las columnas donde correspondan rellenando con None
                    df_anuncio = df_anuncio.reindex(columns=columnas_globales, fill_value=None)

                    #concateno al dataframe intermedio_pag
                    df_intermedio_pag = pd.concat([df_intermedio_pag, df_anuncio], ignore_index=True)

                #concateno al dataframe intermedio_prov
                df_intermedio_prov = pd.concat([df_intermedio_prov, df_intermedio_pag], ignore_index=True)

                #concateno al dataframe final_prov
                df_final_prov = pd.concat([df_final_prov, df_intermedio_prov], ignore_index=True)
                
            except Exception as e:
                print(f"Error en página {pag} de {provincia}: {e}")
                break  # para que pase a la siguiente provincia

        #guardamos los datos al final de cada provincia
        df_intermedio_prov.to_csv(f"../../datos/red_piso/provincias_completas/datos_{provincia}.csv")

finally:
    # cierro el driver siempre al final
    df_final_prov.to_csv(f"../../datos/red_piso/csv_unico/datos_provincias_unico.csv")
    driver.quit()


No se encontraron pisos en la página 1 de la provincia álava.
No se encontraron pisos en la página 2 de la provincia albacete.
No se encontraron pisos en la página 8 de la provincia alicante.
No se encontraron pisos en la página 1 de la provincia almería.
No se encontraron pisos en la página 2 de la provincia ávila.
No se encontraron pisos en la página 1 de la provincia badajoz.
No se encontraron pisos en la página 1 de la provincia baleares.
No se encontraron pisos en la página 6 de la provincia barcelona.
No se encontraron pisos en la página 2 de la provincia burgos.
No se encontraron pisos en la página 1 de la provincia cáceres.
No se encontraron pisos en la página 7 de la provincia cádiz.
No se encontraron pisos en la página 5 de la provincia castellón.
No se encontraron pisos en la página 1 de la provincia ciudad-real.
No se encontraron pisos en la página 2 de la provincia córdoba.
No se encontraron pisos en la página 1 de la provincia a-coruña.
No se encontraron pisos en la págin

In [29]:
madrid=pd.read_csv("../../datos/red_piso/provincias_completas/datos_madrid.csv", index_col=0)
barcelona=pd.read_csv("../../datos/red_piso/provincias_completas/datos_barcelona.csv", index_col=0)

In [30]:
madrid

,Acceso minusválidos,Metros,Plantas,Cal. energética:,Individual,Agua caliente,Consumo,provincia,Exterior,Número de terrazas,Piscina,Pista de tenis,Calefacción,Garajes,Tipo de suelo,Conserje,Pista de fútbol,Gimnasio,Adosado,Año de construcción,Zonas verdes,Armarios,Tipo de fachada,Alarma,Pisos,Permite mascotas,Cal. energética,Ascensor,Campo de golf,Comunidad,Metros útiles,Puerta de seguridad,Aire acondicionado,Pista de pádel,Trastero,Orientación,Pista de squash,Pista de baloncesto,Antigüedad,Pareado,Circuito cerrado de seguridad,Inquilinos,Dormitorios,Baños,Amueblado,Emisiones,Precio,Cocina equipada,Estado,Salida de humos,Unnamed: 51
0,NaN,64.0,NaN,NaN,NaN,Individual,NaN,madrid,Si,NaN,NaN,NaN,NaN,NaN,Tarima normal,NaN,NaN,NaN,NaN,1962.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,Si,NaN,NaN,NaN,NaN,NaN,Más de 50 años,NaN,NaN,NaN,2.0,1.0,NaN,NaN,106500,NaN,Perfecto Estado,NaN,NaN
1,NaN,88.0,5.0,Si,NaN,NaN,NaN,madrid,Si,1.0,NaN,NaN,Eléctrica,NaN,Terrazo,NaN,NaN,NaN,NaN,1978.0,NaN,1.0,NaN,Si,NaN,NaN,NaN,Si,NaN,"70,00 €",70.0,Si,Si,NaN,NaN,NaN,NaN,NaN,Entre 35 y 50 Años,NaN,NaN,NaN,3.0,1.0,NaN,NaN,115000,NaN,Perfecto Estado,NaN,NaN
2,NaN,37.0,2.0,Si,NaN,Individual,111 KW h/m² año,madrid,Si,NaN,NaN,NaN,Eléctrica,NaN,Gres,NaN,NaN,NaN,NaN,1965.0,NaN,1.0,Ladrillo,NaN,6.0,NaN,NaN,NaN,NaN,NaN,34.0,Si,NaN,NaN,NaN,Sur,NaN,NaN,Más de 50 años,NaN,NaN,NaN,1.0,1.0,NaN,18 Kg CO2/m² año,120000,Si,Perfecto Estado,NaN,NaN
3,NaN,33.0,2.0,Si,NaN,Individual,145 KW h/m² año,madrid,NaN,NaN,NaN,NaN,Eléctrica,NaN,Gres,NaN,NaN,NaN,NaN,1965.0,NaN,1.0,Ladrillo,NaN,6.0,NaN,NaN,NaN,NaN,NaN,30.0,Si,NaN,NaN,NaN,Oeste,NaN,NaN,Más de 50 años,NaN,NaN,NaN,1.0,1.0,NaN,24 Kg CO2/m² año,120000,NaN,Perfecto Estado,NaN,NaN
4,Si,75.0,4.0,NaN,NaN,Individual,NaN,madrid,Si,1.0,NaN,NaN,Individual,NaN,Sintasol,NaN,NaN,NaN,NaN,1976.0,NaN,3.0,NaN,NaN,16.0,NaN,NaN,Si,NaN,"70,00 €",69.0,Si,Si,NaN,NaN,Sureste,NaN,NaN,Entre 35 y 50 Años,NaN,NaN,NaN,2.0,1.0,NaN,NaN,125000,NaN,A reformar,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,Si,85.0,NaN,Si,NaN,Individual,NaN,madrid,Si,2.0,NaN,NaN,Central,NaN,Tarima Flotante,Si,NaN,NaN,NaN,1966.0,NaN,NaN,Ladrillo,NaN,NaN,NaN,NaN,Si,NaN,"140,00 €",68.0,NaN,NaN,NaN,NaN,Norte,NaN,NaN,Más de 50 años,NaN,NaN,NaN,2.0,1.0,NaN,NaN,A consultar,NaN,Perfecto Estado,NaN,NaN
2876,NaN,63.0,NaN,Si,NaN,Individual,NaN,madrid,Si,2.0,NaN,NaN,Individual,1.0,Parquet,NaN,NaN,NaN,NaN,1962.0,NaN,NaN,Ladrillo,NaN,NaN,NaN,NaN,Si,NaN,"50,00 €",50.0,NaN,NaN,NaN,NaN,Suroeste,NaN,NaN,Más de 50 años,NaN,NaN,NaN,2.0,1.0,NaN,NaN,A consultar,NaN,Perfecto Estado,NaN,NaN
2877,NaN,85.0,NaN,Si,NaN,Individual,371 KW h/m² año,madrid,Si,NaN,NaN,NaN,Individual,NaN,Parquet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cemento,NaN,NaN,NaN,NaN,Si,NaN,"60,00 €",68.0,NaN,Si,NaN,Si,Suroeste,NaN,NaN,Más de 50 años,NaN,NaN,NaN,3.0,1.0,NaN,78 Kg CO2/m² año,A consultar,NaN,NaN,NaN,NaN
2878,NaN,80.0,NaN,Si,NaN,Individual,NaN,madrid,Si,NaN,NaN,NaN,Central,NaN,Tarima Flotante,NaN,NaN,NaN,NaN,1953.0,NaN,NaN,Ladrillo,NaN,NaN,NaN,NaN,Si,NaN,"100,00 €",64.0,NaN,Si,NaN,NaN,Noreste,NaN,NaN,Entre 35 y 50 Años,NaN,NaN,NaN,3.0,1.0,NaN,NaN,A consultar,NaN,Perfecto Estado,NaN,NaN


In [31]:
barcelona.shape

(51, 46)

In [32]:
df_unico=pd.read_csv("../../datos/red_piso/csv_unico/datos_provincias_unico.csv", index_col=0)

In [33]:
df_unico.shape

(352116, 51)

In [35]:
df_unico[df_unico.duplicated()]

,Ascensor,Metros,Año de construcción,Acceso minusválidos,Metros útiles,Dormitorios,Agua caliente,provincia,Baños,Exterior,Trastero,Orientación,Amueblado,Calefacción,Precio,Antigüedad,Cocina equipada,Número de terrazas,Garajes,Tipo de suelo,Tipo de fachada,Circuito cerrado de seguridad,Puerta de seguridad,Aire acondicionado,Estado,Permite mascotas,Cal. energética:,Comunidad,Piscina,Armarios,Alarma,Plantas,Zonas verdes,Pareado,Pisos,Individual,Pista de tenis,Conserje,Pista de fútbol,Adosado,Pista de baloncesto,Consumo,Inquilinos,Emisiones,Campo de golf,Pista de pádel,Cal. energética,Pista de squash,Gimnasio,Salida de humos,Unnamed: 51
15,NaN,50.0,NaN,Si,40.0,1.0,Individual,alicante,1.0,Si,NaN,NaN,Si,Individual,85000,Entre 35 y 50 Años,Si,NaN,NaN,Gres,Cemento,NaN,NaN,Si,Perfecto Estado,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,64.0,1968.0,NaN,58.0,3.0,Individual,alicante,1.0,Si,Si,Sureste,NaN,Eléctrica,105000,Más de 50 años,NaN,1.0,NaN,Gres,Cemento,NaN,Si,NaN,Perfecto Estado,NaN,Si,"30,00 €",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,Si,100.0,1985.0,Si,85.0,3.0,Individual,alicante,1.0,Si,NaN,Sur,Si,NaN,110000,Entre 35 y 50 Años,Si,1.0,NaN,Baldosa,Ladrillo,Si,Si,Si,Perfecto Estado,NaN,NaN,NaN,Si,1.0,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Si,110.0,NaN,NaN,95.0,3.0,Individual,alicante,1.0,Si,NaN,Norte,Si,NaN,135000,NaN,Si,2.0,NaN,Tarima Flotante,Cemento,Si,Si,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,Si,127.0,1979.0,NaN,105.0,4.0,Individual,alicante,2.0,Si,NaN,Este,Si,NaN,157500,Mas de 25 Años,Si,1.0,NaN,Terrazo,Ladrillo,NaN,Si,NaN,A reformar,NaN,Si,"30,00 €",NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352106,Si,211.0,1974.0,Si,111.0,4.0,Central,zaragoza,2.0,Si,Si,Noreste,NaN,Central,420000,Entre 35 y 50 Años,Si,2.0,1.0,Tarima normal,Ladrillo,Si,Si,NaN,A reformar,NaN,NaN,"90,00 €",NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352107,NaN,205.0,1962.0,NaN,204.0,7.0,NaN,zaragoza,2.0,Si,NaN,NaN,NaN,NaN,A consultar,Más de 50 años,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,A reformar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352108,Si,67.0,2008.0,Si,57.0,2.0,Individual,zaragoza,1.0,Si,Si,Oeste,NaN,Individual,A consultar,Entre 10 y 15 Años,NaN,1.0,2.0,Tarima normal,NaN,NaN,Si,NaN,Perfecto Estado,NaN,NaN,"53,00 €",NaN,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352109,Si,89.0,1976.0,Si,81.0,3.0,Central,zaragoza,2.0,Si,Si,NaN,NaN,Central,A consultar,Entre 35 y 50 Años,NaN,NaN,NaN,Terrazo,NaN,NaN,Si,NaN,A reformar,NaN,NaN,"120,00 €",Si,NaN,NaN,9.0,NaN,NaN,36.0,NaN,NaN,Si,NaN,NaN,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_fin=df_unico.drop_duplicates()
df_fin.shape

(3885, 51)

In [38]:
df_fin.reset_index(drop=True, inplace=True)

In [39]:
df_fin

,Ascensor,Metros,Año de construcción,Acceso minusválidos,Metros útiles,Dormitorios,Agua caliente,provincia,Baños,Exterior,Trastero,Orientación,Amueblado,Calefacción,Precio,Antigüedad,Cocina equipada,Número de terrazas,Garajes,Tipo de suelo,Tipo de fachada,Circuito cerrado de seguridad,Puerta de seguridad,Aire acondicionado,Estado,Permite mascotas,Cal. energética:,Comunidad,Piscina,Armarios,Alarma,Plantas,Zonas verdes,Pareado,Pisos,Individual,Pista de tenis,Conserje,Pista de fútbol,Adosado,Pista de baloncesto,Consumo,Inquilinos,Emisiones,Campo de golf,Pista de pádel,Cal. energética,Pista de squash,Gimnasio,Salida de humos,Unnamed: 51
0,Si,142.0,1974.0,Si,120.0,3.0,Central,albacete,2.0,Si,Si,Oeste,Si,Central,249000,Mas de 25 Años,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Si,137.0,1974.0,Si,136.0,3.0,Individual,albacete,2.0,Si,NaN,Noreste,Si,Individual,250000,NaN,Si,1.0,1.0,Parquet,Piedra,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Si,96.0,1987.0,Si,95.0,3.0,Individual,albacete,2.0,NaN,NaN,NaN,Si,Individual,A consultar,NaN,Si,NaN,NaN,Parquet,Ladrillo,Si,Si,Si,Perfecto Estado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,50.0,NaN,Si,40.0,1.0,Individual,alicante,1.0,Si,NaN,NaN,Si,Individual,85000,Entre 35 y 50 Años,Si,NaN,NaN,Gres,Cemento,NaN,NaN,Si,Perfecto Estado,Si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,64.0,1968.0,NaN,58.0,3.0,Individual,alicante,1.0,Si,Si,Sureste,NaN,Eléctrica,105000,Más de 50 años,NaN,1.0,NaN,Gres,Cemento,NaN,Si,NaN,Perfecto Estado,NaN,Si,"30,00 €",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3880,NaN,71.0,1964.0,NaN,65.0,3.0,Individual,zaragoza,1.0,Si,NaN,Noreste,NaN,Individual,A consultar,Más de 50 años,NaN,3.0,NaN,Baldosa,NaN,NaN,NaN,NaN,A reformar,NaN,NaN,"20,00 €",NaN,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3881,Si,67.0,NaN,NaN,57.0,2.0,Individual,zaragoza,1.0,NaN,NaN,NaN,NaN,Individual,A consultar,Entre 35 y 50 Años,NaN,NaN,NaN,Baldosa,NaN,NaN,NaN,NaN,Perfecto Estado,NaN,NaN,"55,00 €",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3882,Si,70.0,NaN,Si,65.0,2.0,Individual,zaragoza,1.0,NaN,NaN,Sureste,NaN,Central,A consultar,Más de 50 años,Si,1.0,NaN,Baldosa,NaN,NaN,Si,Si,Perfecto Estado,NaN,NaN,"100,00 €",NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3883,NaN,86.0,1954.0,NaN,80.0,3.0,Individual,zaragoza,1.0,NaN,NaN,Sur,NaN,Individual,A consultar,Entre 35 y 50 Años,NaN,NaN,NaN,Tarima Flotante,Ladrillo,NaN,NaN,NaN,Perfecto Estado,NaN,NaN,"35,00 €",NaN,NaN,NaN,4.0,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Siguientes pasos:  
  
- **Los NaN que correspondan a opciones de Si igual hay que pasarlos a No**  

- **Ver si tiene sentido que me haya quedado solo con 3885 (viendo las gráficas de distribución por provincia)**    

In [ ]:
df_fin[df_fin["Unnamed: 51"]== "Si"]  #esta nos la tendremos que cargar

,Ascensor,Metros,Año de construcción,Acceso minusválidos,Metros útiles,Dormitorios,Agua caliente,provincia,Baños,Exterior,Trastero,Orientación,Amueblado,Calefacción,Precio,Antigüedad,Cocina equipada,Número de terrazas,Garajes,Tipo de suelo,Tipo de fachada,Circuito cerrado de seguridad,Puerta de seguridad,Aire acondicionado,Estado,Permite mascotas,Cal. energética:,Comunidad,Piscina,Armarios,Alarma,Plantas,Zonas verdes,Pareado,Pisos,Individual,Pista de tenis,Conserje,Pista de fútbol,Adosado,Pista de baloncesto,Consumo,Inquilinos,Emisiones,Campo de golf,Pista de pádel,Cal. energética,Pista de squash,Gimnasio,Salida de humos,Unnamed: 51
1592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,madrid,NaN,NaN,NaN,NaN,NaN,NaN,A consultar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Si
